In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

In [2]:
data = pd.read_csv("USvideos.csv")

In [3]:
preprocessed_data = data.drop(['description', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'publish_time', 'channel_title', 'trending_date', 'video_id', 'tags', 'title'], axis=1)

In [4]:
preprocessed_data.head()

,category_id,views,likes,dislikes,comment_count
0,22,748374,57527,2966,15954
1,24,2418783,97185,6146,12703
2,23,3191434,146033,5339,8181
3,24,343168,10172,666,2146
4,24,2095731,132235,1989,17518


In [5]:
X = preprocessed_data.drop('category_id', axis=1)
y = preprocessed_data['category_id']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
scaling = MinMaxScaler(feature_range=(0, 1)).fit(X_train)
quantile_scaler = QuantileTransformer(output_distribution='uniform').fit(X_train)
quantile_transformer = QuantileTransformer(output_distribution='uniform').fit(X_train)
# X_train = scaling.transform(X_train)
# X_test = scaling.transform(X_test)
Xq_train = quantile_scaler.transform(X_train)
Xq_test = quantile_scaler.transform(X_test)
Xqn_train = quantile_transformer.transform(X_train)
Xqn_test = quantile_transformer.transform(X_train)

In [20]:
# model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=20, random_state=33)
svc = SVC(kernel='rbf', C=1, decision_function_shape='sigmoid', gamma=1000)
svc.fit(Xqn_train, y_train)
#model.fit(X_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='sigmoid', degree=3, gamma=1000, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [21]:
y_pred = svc.predict(Xq_test)

In [22]:
accuracy_score(y_test, y_pred)

0.7021978021978021

In [56]:
kmodel = KNeighborsClassifier(n_neighbors=32, p=1, weights='distance')
kmodel.fit(Xq_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=32, p=1,
                     weights='distance')

In [59]:
y_predk = kmodel.predict(Xq_test)

In [60]:
accuracy_score(y_test, y_predk)

0.6454212454212455

In [29]:
X_train

array([[1.08717422e-03, 1.58483687e-03, 5.79305073e-05, 3.48859413e-04],
       [2.10765021e-02, 2.41699646e-02, 7.46228545e-03, 2.91051572e-02],
       [7.61426907e-04, 9.35012782e-04, 1.07499910e-04, 6.40432439e-04],
       ...,
       [6.31629289e-03, 4.60149556e-03, 4.44930185e-04, 1.04510936e-03],
       [2.32324634e-02, 6.36321711e-03, 1.91170674e-03, 5.27769209e-03],
       [5.16368236e-04, 1.20541655e-03, 4.47916293e-05, 2.95979671e-04]])

In [25]:
X

,views,likes,dislikes,comment_count
0,748374,57527,2966,15954
1,2418783,97185,6146,12703
2,3191434,146033,5339,8181
3,343168,10172,666,2146
4,2095731,132235,1989,17518
5,119180,9763,511,1434
6,2103417,15993,2445,1970
7,817732,23663,778,3432
8,826059,3543,119,340
9,256426,12654,1363,2368


In [62]:
X['views'].median()

681861.0